In [1]:
import simpy
import random
import pandas as pd

# DataFrames para almacenar datos de recursos y eventos
df_recursos = pd.DataFrame(columns=["ID Trabajador", "Disponible"])
df_eventos = pd.DataFrame(columns=["ID Caja", "Tipo Caja", "Inicio Produccion", "Fin Produccion"])

# Clase para definir el proceso de Produccion de cajas
class Caja:
    def __init__(self, env, tipo, tiempo_produccion, trabajadores, id_caja):
        self.env = env
        self.tipo = tipo
        self.tiempo_produccion = tiempo_produccion
        self.trabajadores = trabajadores
        self.id_caja = id_caja

    def producir(self):
        # Simula el tiempo de Produccion de la caja
        inicio = self.env.now
        yield self.env.timeout(self.tiempo_produccion)
        fin = self.env.now
        # Almacenar el evento de Produccion
        global df_eventos
        df_eventos = df_eventos._append({
            "ID Caja": self.id_caja,
            "Tipo Caja": self.tipo,
            "Inicio Produccion": inicio,
            "Fin Produccion": fin
        }, ignore_index=True)
        print(f"Se produjo una caja {self.tipo} (ID: {self.id_caja}) en el tiempo {fin}")

# Clase para definir la fábrica con los recursos (trabajadores)
class Fabrica:
    def __init__(self, env, num_trabajadores):
        self.env = env
        self.trabajadores = simpy.Resource(env, num_trabajadores)
        self.id_trabajador = 1

        # Inicializar trabajadores en el DataFrame
        global df_recursos
        for i in range(num_trabajadores):
            df_recursos = df_recursos._append({"ID Trabajador": i + 1, "Disponible": True}, ignore_index=True)

    def fabricar_caja(self, tipo, tiempo_produccion, id_caja):
        # Simulación del proceso de fabricación de cajas con trabajadores disponibles
        with self.trabajadores.request() as request:
            yield request
            print(f"Inicio la Produccion de caja {tipo} (ID: {id_caja}) en el tiempo {self.env.now}")
            caja = Caja(self.env, tipo, tiempo_produccion, self.trabajadores, id_caja)
            yield self.env.process(caja.producir())

# Función que genera órdenes de cajas
def generar_ordenes(env, fabrica, tasa_ordenes):
    id_caja = 0
    while True:
        # Se genera un nuevo tipo de caja cada 'tasa_ordenes' tiempo
        tipo_caja = random.choice(["Pequeña", "Mediana", "Grande"])
        tiempo_produccion = random.randint(2, 6)  # Tiempo de Produccion aleatorio
        env.process(fabrica.fabricar_caja(tipo_caja, tiempo_produccion, id_caja))
        id_caja += 1
        yield env.timeout(tasa_ordenes)

# Parámetros de simulación
def correr_simulacion(num_trabajadores, tasa_ordenes, tiempo_simulacion):
    env = simpy.Environment()
    fabrica = Fabrica(env, num_trabajadores)
    env.process(generar_ordenes(env, fabrica, tasa_ordenes))
    env.run(until=tiempo_simulacion)

    # Mostrar DataFrames después de la simulación
    print("\nPerfil de Recursos:")
    print(df_recursos)
    print("\nRegistro de Eventos:")
    print(df_eventos)

# Correr la simulación
correr_simulacion(num_trabajadores=3, tasa_ordenes=4, tiempo_simulacion=100)



#Preguntas 
total_cajas = len(df_eventos)
print(f"Total de cajas producidas: {total_cajas}")   #cuantas cajas

tipo_mas_producido = df_eventos['Tipo Caja'].value_counts().idxmax() 
cantidad_tipo_mas_producido = df_eventos['Tipo Caja'].value_counts().max()
print(f"El tipo de caja más producido fue: {tipo_mas_producido} con {cantidad_tipo_mas_producido} cajas")


df_eventos['Duración Produccion'] = df_eventos['Fin Produccion'] - df_eventos['Inicio Produccion']   #¿Cuál fue el tiempo promedio de Produccion de una caja?
tiempo_promedio = df_eventos['Duración Produccion'].mean()
print(f"El tiempo promedio de Produccion de una caja fue: {tiempo_promedio:.2f} unidades de tiempo")
  
caja_mas_lenta = df_eventos.loc[df_eventos['Duración Produccion'].idxmax()]  #¿Cuál fue la caja que tardó más tiempo en producirse?
print(f"La caja que tardó más tiempo en producirse fue la de ID {caja_mas_lenta['ID Caja']} ({caja_mas_lenta['Tipo Caja']})")


tiempo_total_produccion = df_eventos['Duración Produccion'].sum() #¿Cuál fue el tiempo total de Produccion de todas las cajas?
print(f"El tiempo total de Produccion de todas las cajas fue: {tiempo_total_produccion} unidades de tiempo")






ModuleNotFoundError: No module named 'simpy'